# With the AML network, observe the effects of BCL2 inhibition on patient scores

### Importing BMatrix, BooleanNetwork, and Clinical Data

In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

# import booleanNetwork module from ./src
import sys
sys.path.append('./src/')
import booleanNetwork as bn
import BMatrix  

drug_response_df = pd.read_excel("./input_files/drug_response_df.xlsx",keep_default_na=False)

In [ ]:
drug_response_df

### Getting the variables needed for the simulation and calculation

In [ ]:
file = './input_files/fig3_AML_equations.txt'

equations = BMatrix.get_equations(file)
gene_dict = BMatrix.get_gene_dict(equations)
upstream_genes = BMatrix.get_upstream_genes(equations)

connectivity_matrix = BMatrix.get_connectivity_matrix(equations,upstream_genes,gene_dict)
#print(connectivity_matrix)
#print('\n')

truth_table = BMatrix.get_truth_table(equations,upstream_genes)
#print(truth_table)

ngenes= len(equations)

In [ ]:
file_equations = './input_files/fig3_calculating_equations.txt'

cal_equations = BMatrix.get_equations(file_equations)
cal_upstream_genes = BMatrix.get_cal_upstream_genes(cal_equations)
cal_functions = BMatrix.get_cal_functions(cal_equations)

In [ ]:
mutation_dict = None

file_perturbed = './input_files/perturbed_genes.txt'
perturbed_dict = BMatrix.get_mutation_dict(file_perturbed)
perturbed_genes = 'BCL2'

### Simulation of BCL2 inhibition

In [ ]:
all_profiles = np.array([[profile] for profile in drug_response_df['mutation_profiles']])

In [ ]:
before_inhibition = []
networks = {}

for i in range(len(all_profiles)): 
    profile = all_profiles[i,0] # Gets the profile as a string ex. 'FLT3'
    print(profile)
    
    mutated_connectivity_matrix, x0 = BMatrix.get_knocking_genes(profile, mutation_dict, connectivity_matrix, gene_dict)

    #making the networks and running the simulation
    print('starting simulation')
    network = bn.BooleanNetwork(ngenes, mutated_connectivity_matrix, truth_table, x0)
    network_traj = network.update_noise(0.05, 200000)
    
    #calculating the scores  
    print('getting final score')
    cal_range = network_traj[-100000:]
    
    scores_dict, final_score = BMatrix.get_calculating_scores(network_traj, cal_functions, cal_upstream_genes, gene_dict, cal_range)

    print(final_score)
    
    before_inhibition.append(final_score)
    
print(before_inhibition)

In [ ]:
after_inhibition = []
networks = {}

for i in range(len(all_profiles)): 
    profile = all_profiles[i,0] # Gets the profile as a string ex. 'FLT3'
    print(profile)
    
    mutated_connectivity_matrix, x0 = BMatrix.get_knocking_genes(profile, mutation_dict, connectivity_matrix, gene_dict, perturbed_genes, perturbed_dict)

    #making the networks and running the simulation
    print('starting simulation')
    network = bn.BooleanNetwork(ngenes, mutated_connectivity_matrix, truth_table, x0)
    network_traj = network.update_noise(0.05, 200000)
    
    #calculating the scores  
    print('getting final score')
    cal_range = network_traj[-100000:]
    
    scores_dict, final_score = BMatrix.get_calculating_scores(network_traj, cal_functions, cal_upstream_genes, gene_dict, cal_range)

    print(final_score)
    
    after_inhibition.append(final_score)
    
print(after_inhibition)

In [ ]:
labId = drug_response_df['labId'][0:len(all_profiles)]

scores_df = pd.DataFrame({
    'labId': labId,
    'no_inhibition_score': before_inhibition,
    'BCL2_inhibition_score': after_inhibition
})

scores_df['score_diff'] = scores_df.apply(lambda x: x['no_inhibition_score'] - x['BCL2_inhibition_score'], axis=1) 

In [ ]:
BCL2_inhibition_df = pd.merge(drug_response_df, scores_df, on='labId', how='outer')

BCL2_inhibition_df